# Imports

In [1]:
import pandas as pd
import numpy as np
import psycopg2
import pandas as pd
import pandas.io.sql as psql
from sqlalchemy import create_engine, text
import yaml
from pathlib import Path
import os

project_dir = Path('__main__').resolve().parent.parent

# Conexão com ElephantSQL

In [2]:
db_params = {
    'host': 'kesavan.db.elephantsql.com',
    'database': 'qqvrbfda',
    'user': 'qqvrbfda',
    'password': '6M-92F_CHXXo86yhk7s6oIABSqmkSTMP'
}

In [3]:
#Criando conexão
conn = psycopg2.connect(
    host=db_params['host'],
    database=db_params['database'],
    user=db_params['user'],
    password=db_params['password']
)

# Criação de Tabela

In [4]:
# Create a cursor object
cur = conn.cursor()

# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

In [11]:
# Criar tabela
cur.execute('''
    CREATE TABLE churn_index (
        churn_index_id SERIAL PRIMARY KEY,
        customer_id INT,
        credit_score INT,
        country VARCHAR(50),
        gender VARCHAR(10),
        age INT,
        tenure INT,
        balance FLOAT,
        products_number INT,
        credit_card INT,
        active_member INT,
        estimated_salary FLOAT,
        churn FLOAT
    );
''')

In [12]:
#Fechando conexão, boa prática
cur.close()
conn.close()

# Inserção de Dados

In [5]:
#Definindo a engine do banco de dados
engine = create_engine(f'postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}/{db_params["database"]}')

In [22]:
# Lendo os dados
df = pd.read_csv('Bank Customer Churn PredictionValidated.csv')
df.head()

,index,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1.0
1,1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0
2,2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1.0
3,3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0.0
4,4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0.0


In [23]:
df.to_sql('churn_index', engine, if_exists='replace', index=False)

500

# SQL

## Comandos

In [24]:
# Create a cursor object
cur = conn.cursor()

# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

### SELECT, FROM, WHERE

In [27]:
tudo = '''
SELECT
  *
FROM churn_index
'''

In [26]:
df_result = psql.read_sql(tudo, conn)
df_result

C:\Users\lucas\AppData\Local\Temp\ipykernel_15640\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,index,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1.0
1,1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0
2,2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1.0
3,3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0.0
4,4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,10495,15662483,850,France,Male,43,7,0.00,2,1,1,173851.11,0.0
10496,10496,15699340,680,France,Male,37,4,0.00,2,1,0,61240.87,0.0
10497,10497,15668594,620,Germany,Female,25,1,137712.01,1,1,1,76197.05,0.0
10498,10498,15750396,670,France,Male,33,1,0.00,2,1,1,86413.11,0.0


In [67]:
query = '''
SELECT
  url_length,
  n_dots,
  n_hypens,
  n_underline,
  n_slash,
  n_questionmark,
  phishing
FROM PHISHING_DATA
WHERE url_length >= 1000 AND n_slash > 0
'''

In [71]:
query = '''
SELECT
  url_length,
  n_dots,
  n_hypens,
  n_underline,
  n_slash,
  n_questionmark,
  phishing
FROM PHISHING_DATA
WHERE n_questionmark > 3 OR n_slash > 2
LIMIT 10
'''

### COUNT, AVG, SUM, MAX, GROUP BY, ORDER BY

In [81]:
query = '''
SELECT
  phishing,
  COUNT(url_length) as qtd_linhas
FROM PHISHING_DATA
GROUP BY phishing
'''

In [85]:
query = '''
SELECT
  phishing,
  COUNT(url_length) as qtd_linhas,
  AVG(url_length) as media_tamanho_url,
  SUM(url_length) as soma_tamanho_urls
FROM PHISHING_DATA
GROUP BY phishing
ORDER BY AVG(url_length)
'''

In [87]:
query = '''
SELECT
  phishing,
  COUNT(url_length) as qtd_linhas,
  AVG(url_length) as media_tamanho_url,
  SUM(url_length) as soma_tamanho_urls
FROM PHISHING_DATA
GROUP BY phishing
ORDER BY COUNT(url_length) DESC
'''

In [91]:
query = '''
SELECT
  phishing,
  n_hypens,
  COUNT(url_length) as qtd_linhas,
  AVG(url_length) as media_tamanho_urls,
  SUM(url_length) as soma_tamanho_urls,
  MAX(url_length) as tamanho_maximo_url
FROM PHISHING_DATA
WHERE n_hypens <= 2
GROUP BY phishing, n_hypens
ORDER BY COUNT(url_length) DESC
'''

# 

### CASE WHEN

In [93]:
query = '''
SELECT
  CASE WHEN URL_LENGTH < 100 then 'PEQUENA'
       WHEN URL_LENGTH < 400 then 'MEDIA'
       WHEN URL_LENGTH < 600 then 'GRANDINHA'
      ELSE 'GRANDE'
  END AS TAMANHO_URL,
  URL_LENGTH,
  PHISHING
FROM PHISHING_DATA
LIMIT 10
'''

## Exemplos

In [24]:
# Create a cursor object
cur = conn.cursor()

# Set automatic commit to be true, so that each action is committed without having to call conn.committ() after each command
conn.set_session(autocommit=True)

In [95]:
# Verificando quantas linhas tenho na minha tabela, quantidade de links de
#phishing e o percentual total de links de phishing na minha tabela
query = '''
SELECT
  COUNT(*) AS QTD_LINHAS,
  SUM(PHISHING) AS QTD_LINKS_PHISHING,
  AVG(PHISHING) AS PERC_LINKS_PHISHING
FROM PHISHING_DATA
'''

In [96]:
df_result = psql.read_sql(query, conn)

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1564066013.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


In [97]:
df_result

,qtd_linhas,qtd_links_phishing,perc_links_phishing
0,100077,35575.0,0.362733


In [98]:
# Verificando quantidade de phishings e percentual baseado no tamanho da url
query = '''
SELECT
  URL_LENGTH,
  COUNT(*) AS QTD_LINHAS,
  SUM(PHISHING) AS QTD_PHISHING,
  AVG(PHISHING) AS PERC_PHISHING
FROM PHISHING_DATA
GROUP BY URL_LENGTH
'''

In [100]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,url_length,qtd_linhas,qtd_phishing,perc_phishing
0,938,1,1.0,1.000000
1,184,6,6.0,1.000000
2,87,136,108.0,0.812030
3,477,2,2.0,1.000000
4,273,9,9.0,1.000000
...,...,...,...,...
485,536,1,1.0,1.000000
486,449,1,1.0,1.000000
487,64,500,412.0,0.839104
488,55,485,353.0,0.752665


In [101]:
# Agrupando o tamanho das url em pequenas, médias e grandes, e verificando as métricas
query = '''
SELECT
  CASE WHEN URL_LENGTH < 100 then 'PEQUENA'
       WHEN URL_LENGTH < 400 then 'MEDIA'
      ELSE 'GRANDE'
  END AS TAMANHO_URL,
  COUNT(*) AS QTD_LINHAS,
  SUM(PHISHING) AS QTD_PHISHING,
  AVG(PHISHING) AS PERC_PHISHING
FROM PHISHING_DATA
GROUP BY 1
ORDER BY COUNT(*)
'''

In [102]:
df_result = psql.read_sql(query, conn)
df_result

C:\Users\diego\AppData\Local\Temp\ipykernel_20224\1607457878.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(query, conn)


,tamanho_url,qtd_linhas,qtd_phishing,perc_phishing
0,GRANDE,149,141.0,0.959184
1,MEDIA,5878,5288.0,0.920613
2,PEQUENA,94050,30146.0,0.327020


## Pra vocês praticarem

Links de apoio: 
* https://learnsql.com/blog/sql-basics-cheat-sheet/
* 

## Dataset de Churn

Utilize somente SQL para responder as perguntas abaixo


*   Qual o total de clientes, total de clientes que realizaram churn e percentual churns tem na sua tabela?
*   Qual os top 2 paises com maior concentração de churn?
*   Em quais paises estão concentrados 90% de todos os clientes que tiveram churn?
*   Qual a idade média e o balanço médio dos clientes que tiveram churn?E dos que não tiveram?
*   Qual o total de clientes, total de churns e percentual de churn para as três faixas de renda a seguir [0-1000); [1000-3500); [3500-infinito)?
*   Qual o percentual de churn por faixas de idade?
*   Qual a mediana, p75, p90 e p99 da renda dos clientes que realizam churn?
*   O Score de Crédito é uma boa variavel para predizer a probabilidade do churn?
*   Acumulando a idade a partir de 18 anos, com quantos anos temos 50% de churn da sua tabela?

In [28]:
df_result = psql.read_sql(tudo, conn)
df_result

C:\Users\lucas\AppData\Local\Temp\ipykernel_15640\245751791.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = psql.read_sql(tudo, conn)


,index,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1.0
1,1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0.0
2,2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1.0
3,3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0.0
4,4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,10495,15662483,850,France,Male,43,7,0.00,2,1,1,173851.11,0.0
10496,10496,15699340,680,France,Male,37,4,0.00,2,1,0,61240.87,0.0
10497,10497,15668594,620,Germany,Female,25,1,137712.01,1,1,1,76197.05,0.0
10498,10498,15750396,670,France,Male,33,1,0.00,2,1,1,86413.11,0.0


In [49]:
#Qual o total de clientes, total de clientes que realizaram churn e percentual churns tem na sua tabela?
query1= '''
    select count(*) as total, avg(churn) as porcentagem, sum(churn) as quantos_churn
    from churn_index

'''

resultado = psql.read_sql(query1, conn)
resultado

C:\Users\lucas\AppData\Local\Temp\ipykernel_15640\4037277360.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado = psql.read_sql(query1, conn)


,total,porcentagem,quantos_churn
0,10500,0.202636,2106.0


In [62]:
#*   Qual os top 2 paises com maior concentração de churn?

query2 = '''
    select country,
    sum(churn) as quantos_churn
    from churn_index
    group by country
    order by quantos_churn desc
    limit 2
'''
resultado2 = psql.read_sql(query2, conn)
resultado2

C:\Users\lucas\AppData\Local\Temp\ipykernel_15640\920258529.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado2 = psql.read_sql(query2, conn)


,country,quantos_churn
0,Germany,821.0
1,France,817.0


In [35]:
#Em quais paises estão concentrados 90% de todos os clientes que tiveram churn?

query3 = '''
    
    select country, porcentagem, sum(porcentagem) OVER(order by porcentagem) from (
    select 
        country,
        sum(churn)/(select sum(churn) from churn_index) as porcentagem
    from churn_index
    group by country
    order by porcentagem desc) as tabela

'''

resultado3 = psql.read_sql(query3,conn)
resultado3

C:\Users\lucas\AppData\Local\Temp\ipykernel_18408\3218013611.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado3 = psql.read_sql(query3,conn)


,country,porcentagem,sum
0,É O BRASA,0.022317,0.022317
1,Spain,0.199905,0.222222
2,France,0.387939,0.610161
3,Germany,0.389839,1.000000


In [73]:
#*   Qual a idade média e o balanço médio dos clientes que tiveram churn?E dos que não tiveram?
query4 = '''
select avg(age) as idade_media, avg(balance) as balanco, avg(churn) as churn from churn_index
where churn>0
union
select avg(age) as idade_media2, avg(balance) as balanco2, avg(churn) as churn from churn_index
where churn = 0
'''

resultado4 = psql.read_sql(query4, conn)
resultado4

C:\Users\lucas\AppData\Local\Temp\ipykernel_15640\3536139045.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado4 = psql.read_sql(query4, conn)


,idade_media,balanco,churn
0,37.411729,72969.311662,0.0
1,44.846154,90967.039397,1.0


In [75]:
#*   Qual o total de clientes, total de churns e percentual de churn para as três faixas de renda a seguir [0-1000); [1000-3500); [3500-infinito)?
query5 = '''
SELECT
  CASE WHEN (estimated_salary >=0) AND (estimated_salary<1000)  then 'Peq'
       WHEN (estimated_salary >= 100) AND (estimated_salary<3500) then 'Med'
      ELSE 'Gra'
  END AS renda,
  COUNT(*) AS clientes,
  SUM(churn) AS qtd_churn,
  AVG(churn) AS perc_churn
FROM churn_index
GROUP BY 1
ORDER BY COUNT(*)
'''

resultado5 = psql.read_sql(query5, conn)
resultado5

C:\Users\lucas\AppData\Local\Temp\ipykernel_15640\1996877724.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado5 = psql.read_sql(query5, conn)


,renda,clientes,qtd_churn,perc_churn
0,Peq,63,13.0,0.206349
1,Med,133,20.0,0.150376
2,Gra,10304,2073.0,0.203295


In [12]:
# Qual o percentual de churn por faixas de idade?

query6 = '''
SELECT
  CASE WHEN (age >=0) AND (age<18)  then 'jovem'
       WHEN (age >= 18) AND (age<28) then 'jovem-adulto 18-28'
       WHEN (age >= 28) AND (age<40) then 'adulto-adulto 28-40'
       WHEN (age >= 40) AND (age<60) then 'jovem-adultão 40-60'
       WHEN (age >= 60) AND (age<200) then 'adulto-adultão 60-200'
      ELSE 'idade invalida'
  END AS faixas,
  AVG(churn) AS perc_churn
FROM churn_index
GROUP BY 1
ORDER BY perc_churn desc
'''

resultado = psql.read_sql(query6, conn)
resultado

C:\Users\lucas\AppData\Local\Temp\ipykernel_2096\2087941171.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado = psql.read_sql(query6, conn)


,faixas,perc_churn
0,jovem-adultão 40-60,0.371043
1,adulto-adultão 60-200,0.270872
2,adulto-adulto 28-40,0.104163
3,jovem-adulto 18-28,0.070076


In [131]:
#Qual a mediana, p75, p90 e p99 da renda dos clientes que realizam churn?

query7 = '''
SELECT COUNT(*) FROM churn_index where churn = 1 
'''

resultado7 = psql.read_sql(query7, conn)
pontoMedio = resultado7/2 - 1

query = '''
SELECT sum(estimated_salary)/2 as mediana FROM(
SELECT * FROM churn_index
WHERE churn = 1 
ORDER BY estimated_salary
OFFSET 1052 LIMIT 2) as tabela
'''


resultado = psql.read_sql(query,conn)
print(resultado)

query2 = '''
SELECT * FROM(
SELECT estima, somatorio/(SELECT sum(estimated_salary) 
FROM churn_index WHERE churn = 1) as porcacumulada
FROM (
(SELECT estima, sum(estima) OVER (order by (estima)) as somatorio from(
    SELECT estimated_salary as estima FROM churn_index WHERE churn = 1 ORDER BY estimated_salary) as tabela)) as tabela2) as t3
WHERE (porcacumulada BETWEEN 0.75 AND 0.7509) OR (porcacumulada BETWEEN 0.90 AND 0.9009) OR (porcacumulada BETWEEN 0.99 AND 0.9909)  

'''
query2res = psql.read_sql(query2,conn)
print(query2res)

     mediana
0  102610.23
      estima  porcacumulada
0  174205.22       0.750605
1  189742.78       0.900138
2  198810.65       0.990687


C:\Users\lucas\AppData\Local\Temp\ipykernel_18408\2139560997.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado7 = psql.read_sql(query7, conn)
C:\Users\lucas\AppData\Local\Temp\ipykernel_18408\2139560997.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado = psql.read_sql(query,conn)
C:\Users\lucas\AppData\Local\Temp\ipykernel_18408\2139560997.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query2res = psql.read_sql(query2,conn)


In [22]:
#O Score de Crédito é uma boa variavel para predizer a probabilidade do churn?
query8 = '''
SELECT
  CASE WHEN (credit_score >=200) AND (credit_score<400)  then '[200-400)'
       WHEN (credit_score >= 400) AND (credit_score<600) then '[400-600)'
       WHEN (credit_score>=600) AND (age<800) then '[600-800)'
       WHEN (credit_score>=800) AND (age<=1000) then '[800-1000]'
      ELSE 'score fora'
  END AS faixas,
  count(churn) as contagem
FROM churn_index
GROUP BY 1
ORDER BY contagem desc
'''
resultado8=psql.read_sql(query8,conn)
resultado8

#aparentemente quanto mais alto o score, mais alto é o chunr

C:\Users\lucas\AppData\Local\Temp\ipykernel_2096\204842304.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado8=psql.read_sql(query8,conn)


,faixas,contagem
0,[600-800),7231
1,[400-600),3141
2,[200-400),21


In [21]:
# Acumulando a idade a partir de 18 anos, com quantos anos temos 50% de churn da sua tabela?

query9='''
SELECT * FROM(
SELECT 
    age, 
    porcentagem, 
    sum(porcentagem) OVER(ORDER BY age) as acumulo 
FROM(
    SELECT 
        age,
        sum(churn)/(select sum(churn) from churn_index) as porcentagem
    FROM churn_index
    GROUP BY age
    ORDER BY age) as tabela1) as tabela2
WHERE acumulo <= 0.5
'''

resultado9 = psql.read_sql(query9,conn)
resultado9

C:\Users\lucas\AppData\Local\Temp\ipykernel_12456\4217149180.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  resultado9 = psql.read_sql(query9,conn)


,age,porcentagem,acumulo
0,18,0.000950,0.000950
1,19,0.000475,0.001425
2,20,0.000950,0.002374
3,21,0.001425,0.003799
4,22,0.005698,0.009497
5,23,0.002849,0.012346
6,24,0.007123,0.019468
7,25,0.003324,0.022792
8,26,0.006648,0.029440
9,27,0.005698,0.035138
